In [55]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [56]:
data = """The cat sat on the mat.
          It was a lovely sunny day.
          Suddenly, the sky turned dark."""

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
vocab_size = len(tokenizer.word_index) + 1

In [64]:
sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

In [65]:
max_sequence_len = max([len(seq) for seq in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

In [66]:
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [67]:
# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.7734 - accuracy: 0.1429
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 2.7710 - accuracy: 0.1429
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 2.7685 - accuracy: 0.1429
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 2.7660 - accuracy: 0.1429
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 2.7634 - accuracy: 0.1429
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 2.7608 - accuracy: 0.1429
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 2.7579 - accuracy: 0.1429
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 2.7549 - accuracy: 0.1429
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 2.7517 - accuracy: 0.1429
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 2.7482 - accuracy: 0.1429
Epoch 11/100
1/1 [===

In [69]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

generated_text = generate_text("The", 10, model, max_sequence_len)
print(generated_text)


The the the the mat mat mat dark dark sunny sunny
